В рамках этого заадания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

Мы будем работать по заранее определённому пайплайну:

1) Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 3 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

2) Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)

3) Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4) Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

5) Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

Логичность и чистота кода (1 балл)

Да, тут 12 баллов - два можно потерять.

В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

### Анна Смирнова БКЛ181


In [1]:
from html import unescape
from bs4 import BeautifulSoup
import re
import time
import html
import collections
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False)
stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
session = requests.session()

функция, которая достает хорошие, плохие и все отзывы о Сумерках с соответствующих страничек:

In [2]:
def get_review(url):
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    page = soup.find_all('span', {'itemprop': 'reviewBody'})
    return page
good = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/good/perpage/75/')
bad = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/bad/perpage/75/')
both = get_review('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/all/perpage/25/')

функция очищает текст от лишних знаков, символов новой строки, тегов и тд

In [3]:
def clean_text(raw):
    lines = []
    for line in raw:
        line = str(line)
        text = re.sub('<.*?>','',line)
        text2 = re.sub('[\n\r]',' ',text)
        lines.append(text2)
    return lines

In [4]:
clean_text(good) #тут можно посмотреть, каким вышел очищенный текст

["Единственная мысль, которая возникает во время просмотра данного фильма, исключая красивый и привлекательный антураж, выдержанные сцены в специфическом для данной картины колорите, это тема романтической любви, коими прототипами являются Ромео и Джульета (книга данного произведения даже появляется во 2 части и упоминается на уроке литературы), а также различные произведения эпохи рыцарских походов аля Дон Кихот, в которых тема любви раскрывается через пожертвование чем-то ради другого, близкого тебе человека, вплоть до самой смерти, тоже самое мы можем лицезреть и в данном фильме, герои буквально влюбляются в друг друга с одного взгляда, их чувства в самом начале настолько скованны, что кажется, как будто актеры плохо сыграли, но, как мне видится, в этом и была задумка режиссерской команды. В конечном итоге, как и во всех мелодрамах, герои не могут оставаться друг без друга, стоит им лишь разлучиться, как их сразу же кидает в морально-духовные переживания переходящие в депресуху, и н

функция токенизирует слова, приводит их к нижнему регистру и возвращает начальные формы

In [5]:
def tokenize(text):
    words = []
    for i in text:
        for word in nltk.word_tokenize(i.lower()):
            if word.isalpha() and word not in stopwords:
                w = morph.parse(word)[0]
                w = w.normal_form
                words.append(w)
    return words

In [6]:
good_list = tokenize(clean_text(good)) #создала новую переменную, чтобы в дальнейшем к ней обращаться
print(good_list)

['единственный', 'мысль', 'который', 'возникать', 'время', 'просмотр', 'данный', 'фильм', 'исключая', 'красивый', 'привлекательный', 'антураж', 'выдержать', 'сцена', 'специфический', 'дать', 'картина', 'колорит', 'это', 'тема', 'романтический', 'любовь', 'кой', 'прототип', 'являться', 'ромео', 'джульета', 'книга', 'данный', 'произведение', 'появляться', 'часть', 'упоминаться', 'урок', 'литература', 'также', 'различный', 'произведение', 'эпоха', 'рыцарский', 'поход', 'аля', 'дон', 'кихот', 'который', 'тема', 'любовь', 'раскрываться', 'пожертвование', 'ради', 'другой', 'близкий', 'ты', 'человек', 'вплоть', 'самый', 'смерть', 'самый', 'мочь', 'лицезреть', 'дать', 'фильм', 'герой', 'буквально', 'влюбляться', 'друг', 'друг', 'один', 'взгляд', 'чувство', 'самый', 'начало', 'настолько', 'скованный', 'кажется', 'актёр', 'плохо', 'сыграть', 'видеться', 'задумка', 'режиссёрский', 'команда', 'конечный', 'итог', 'мелодрама', 'герой', 'мочь', 'оставаться', 'друг', 'друг', 'стоить', 'лишь', 'разлучи

In [7]:
bad_list = tokenize(clean_text(bad))
print(bad_list)

['первое', 'бросаться', 'глаз', 'мрак', 'беспросветный', 'безысходный', 'мрак', 'прекрасно', 'передавать', 'весь', 'наполнение', 'фильм', 'который', 'кишеть', 'кровопийца', 'томный', 'взгляд', 'весь', 'мрачный', 'пафос', 'самый', 'дело', 'очень', 'странно', 'режиссёр', 'продюсер', 'решить', 'пойти', 'шаг', 'выпустить', 'свой', 'суть', 'фантастический', 'мелодрама', 'максимум', 'телевизионный', 'уровень', 'большой', 'экран', 'заход', 'красавчик', 'паттинсон', 'известный', 'четырнадцатилетный', 'публика', 'отшуметь', 'кубок', 'огонь', 'свой', 'дело', 'сделать', 'обеспечить', 'неплохой', 'кассовый', 'сбор', 'заработать', 'награда', 'скупать', 'сюжетный', 'линия', 'повествовать', 'излишний', 'аморфный', 'девушка', 'изабелла', 'кристен', 'стюарт', 'который', 'переезжать', 'матерь', 'пмж', 'свой', 'папаша', 'копа', 'притянуть', 'причина', 'смена', 'дислокация', 'южный', 'аризона', 'затерянный', 'северный', 'пасмурный', 'штат', 'очевидный', 'новенький', 'сразу', 'влюбляться', 'местный', 'эдва

дальше я нашла пересечения списков, чтобы исключить слова, встречающиеся и в положит и в отриц отзывах


In [8]:
same_words = list(set(good_list)&set(bad_list))
print(same_words)

['иметься', 'актриса', 'сестра', 'охота', 'удивительно', 'тип', 'бёрка', 'интересный', 'сага', 'написание', 'определённо', 'вырасти', 'битва', 'любовь', 'успех', 'похожий', 'одержимость', 'прямо', 'размер', 'комментарий', 'легко', 'пить', 'превращаться', 'подняться', 'итого', 'подобрать', 'шея', 'понять', 'семейный', 'цвет', 'совершенно', 'существо', 'атмосфера', 'парочка', 'страна', 'неповторимый', 'нагрузка', 'приехать', 'кендрик', 'отчётливо', 'грим', 'никак', 'оказаться', 'поначалу', 'несомненно', 'дочка', 'краска', 'забывать', 'фантастический', 'конкретный', 'пресный', 'разряд', 'развод', 'появление', 'знакомый', 'подача', 'среднестатистический', 'кома', 'отличие', 'ох', 'такой', 'юмор', 'смеяться', 'постер', 'десятка', 'мелочь', 'год', 'заставить', 'клиповый', 'увидеть', 'едва', 'подхватить', 'вещий', 'волшебник', 'жутко', 'открытый', 'лирический', 'умолчать', 'изыск', 'чётко', 'удивлять', 'создавать', 'дорога', 'пункт', 'читатель', 'дорогой', 'последующий', 'нежность', 'работа',

эта функция убирает из списков слова, которые встречаются в списке выше, а еще убирает шум (я тут убрала слова, встречающиеся только 1 раз, чтобы осталось побольше данных)

In [9]:
def spisok(some_list):
    c = collections.Counter()
    new_list = []
    for i in some_list:
        if i in same_words:
            continue
        else:
            c[i] += 1
    for k,v in c.items():
        if v > 1:
            new_list.append(k)
    return(new_list)

            

In [10]:
ok = spisok(good_list) #новая переменная для обозначения списка со словами из положительных отызывов
print(ok)

['антураж', 'фэнтезийный', 'ключ', 'нотка', 'неоднозначный', 'пошлость', 'отрешённый', 'композитор', 'картер', 'серебристый', 'туман', 'бушевать', 'великолепно', 'выполнить', 'уважение', 'привыкнуть', 'бывать', 'старт', 'фанатизм', 'очарование', 'цветок', 'ура', 'скептицизм', 'переполох', 'поднять', 'ненавистник', 'пыл', 'увлекаться', 'господин', 'правдоподобно', 'обыденный', 'грубость', 'сравнить', 'кровавый', 'однозначно', 'максимально', 'приблизить', 'переработать', 'авторский', 'импонировать', 'прочтение', 'адаптация', 'намного', 'обворожительный', 'карьера', 'привычный', 'издавна', 'кэм', 'качественно', 'hole', 'негативный', 'здравствовать', 'джордж', 'мила', 'сражаться', 'мелодраматизм', 'низкий', 'земной', 'экземпляр', 'пройтись', 'бёруэлла', 'адаптироваться', 'налаживать', 'внимательно', 'заметно', 'сослуживица', 'аналог', 'яркий', 'мелисса', 'розенберг', 'придраться', 'побеждать', 'страдание', 'располагать', 'захотеться', 'смешанный', 'заворожить', 'замечание', 'тонкий', 'хихи

In [11]:
neok = spisok(bad_list) #и для отрицательных
print(neok)

['мрак', 'томный', 'пафос', 'скупать', 'аморфный', 'притянуть', 'местный', 'неудачно', 'получаться', 'голый', 'сопровождаться', 'тематический', 'слабенький', 'вопиять', 'достигнуть', 'реклама', 'собственный', 'подружка', 'мейер', 'забросить', 'идиотский', 'жанровый', 'песочница', 'поиграть', 'дёшево', 'следовательно', 'полиция', 'упоминать', 'ложный', 'центр', 'заимствовать', 'физический', 'совершенный', 'ценить', 'пустота', 'вырвать', 'мгновенный', 'реалистичный', 'пушистый', 'злобный', 'спасение', 'прятаться', 'впечатляющий', 'зал', 'бровь', 'заполнить', 'фейл', 'бездарный', 'киноплёнка', 'коммерческий', 'богомерзкий', 'сиквеловый', 'смотрибельный', 'таковой', 'бревно', 'реализм', 'психолог', 'бочка', 'далее', 'тачка', 'девчачий', 'пятисекундный', 'нападение', 'перерыв', 'часы', 'неестественно', 'двухчасовой', 'блёстка', 'дословно', 'сладенький', 'выработать', 'посредственный', 'бесить', 'вериться', 'мучить', 'возле', 'вспоминаться', 'улечься', 'подрасти', 'наглядно', 'лаж', 'крис', 

функция, определяющая сама, положительный отзыв или отрицательный 

In [12]:
def detect(both):
    res = []
    cleaned = clean_text(both)
    for j in cleaned:
        good = 0
        bad = 0
        tok = tokenize(j)
        for i in tok:
            if i in ok:
                good += 1
            if i in neok:
                bad += 1
        if good > bad:
            print(j[-100:] + ' положительный отзыв ))))')
            res.append('good')
        else:
            print(j[-100:] + ' отрицательный отзыв ((((')
            res.append('bad')
    return(res)

на вход функции подается неочищенный текст с рандомными отзывами, функция их очищает и определяет их тональность

In [13]:
detect(both)

й.   Хоть и в некоторых моментах присутствовали нотки нереалистичности, мне понравился данный фильм. положительный отзыв ))))
евочку, которая строит себе иллюзии касательно лучшей жизни. И кино, собственно, такое же.   4 из 10 положительный отзыв ))))
о сняли такое отстойное кино, чтобы задушить франшизу в младенчестве? Ешкин кот! И здесь оплошали... положительный отзыв ))))
 ноги кормят, и включить ребёнка, который верит, что любовь бывает просто так, ни за что.    6 из 10 положительный отзыв ))))
ер в «ох» при виде красавцев, а так же «эх» при лицезрении зарождений отношений у героев.    7 из 10 положительный отзыв ))))
 не тратьте время и почитайте лучше Гордость и Предубеждение, вот там действительно история любви... положительный отзыв ))))
особое жанровое направление. Но даже не в содержании проблема. Фильм снят и подан плохо.    1 из 10  положительный отзыв ))))
 сказке, окончится хеппи-эндом, а вот потеряное очарование вам уже не удасться возвратить.   9 из 10 положительный отз

['good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'good',
 'bad',
 'good',
 'good',
 'bad',
 'good']

In [14]:
from sklearn.metrics import accuracy_score
results = []
gold = []
for i in good:
    gold.append('good')
for i in bad:
    gold.append('bad')
results = detect(good) + detect(bad)


й.   Хоть и в некоторых моментах присутствовали нотки нереалистичности, мне понравился данный фильм. положительный отзыв ))))
 ноги кормят, и включить ребёнка, который верит, что любовь бывает просто так, ни за что.    6 из 10 положительный отзыв ))))
 сказке, окончится хеппи-эндом, а вот потеряное очарование вам уже не удасться возвратить.   9 из 10 положительный отзыв ))))
 смотреть не советую.   Я же невзыскательный зритель, к тому же девушка, поэтому от меня,   10 из 10 положительный отзыв ))))
раматизм. Но всё шатко. На грани фола. Во второй части так может уже и не прокатить.    На этом всё… положительный отзыв ))))
ился в вампиров. Каллины — Боги вампиров!    «Сумерки» надолго выбили меня из реальности.   10 из 10 положительный отзыв ))))
ию.    Я про то, что непервый просмотр хорошо пошел и прошел, и даже захотелось похвалить «Сумерки». положительный отзыв ))))
ican Mouth.   Думаю, что первая часть была самой лучшей и нужно было на ней остановиться.   10 из 10 положительный отз

In [15]:
print("Accuracy: %.4f" % accuracy_score(results, gold))

Accuracy: 0.5000


предложения по улучшению программы: 
1) взять побольше отзывов, тк на малом количестве отзывов очень много слов, которые встречаются и в положительных, и в отрицательных отзывах, много "шума", так что при удалении всего этого, остается достаточно мало слов для определения тональность
2) было бы неплохо все в одну функцию скрепить, чтобы получилось более последовательно и понятно ))))
3) можно было сделать инпут, чтобы пользователь сам вводил любой отзыв и получал результат (я так делать не стала, но как опция)
